In [28]:
import numpy as np
import pandas as pd
import glob 
import nltk
import re
import string
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
import ast

Remove stopwords?
Combine all for points?
What if word not exist?

### Getting Word Embeddings, Attention Vectors, and Matching Vectors using GloVe

In [3]:
embedding_index = {}

f = open('glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:],dtype='float32')
    embedding_index[word] = coefs
f.close()

print('found word vecs: ',len(embedding_index))

found word vecs:  400000


In [4]:
main_points =pd.read_csv('DebateStar/Meta Data/metadata_appended_main_points.csv') 
main_points.head()

,id,title,date,for,against,For_Main_Points,against_Main_Points
0,d20191112,Capitalism Is a Blessing,2019-11-12,"['John Mackey', 'Katherine Mangu-Ward']","['Bhaskar Sunkara', 'Richard D. Wolff']",['By promoting market competition and rewardin...,['Capitalism serves the interests of large cor...
1,d20191029,Parenting Is Overrated,2019-10-29,"['Robert Plomin', 'Nancy Segal']","['Paige Harden', 'Ann Pleshette Murphy']","[""We're in the midst of a DNA revolution: Whil...","['While DNA is important, factors like familia..."
2,d20191022,Europe Has Declared War on American Tech Compa...,2019-10-22,"['Roslyn Layton', 'Berin Szóka']","['Marietje Schaake', 'Ramesh Srinivasan']",['European regulators have declared war on Ame...,['Brussels isn’t waging war on Silicon Valley....
3,d20190917,Replace Private Insurance with Medicare for All,2019-09-17,"['Dr. Adam Gaffney', 'Joseph Sanberg']","['Nick Gillespie', 'Sally Pipes']",['The United States government should follow t...,['Individuals should have the freedom to choos...
4,d20190912,Unresolved: Shifting Power in the Middle East,2019-09-12,"['Michael Doran', 'Reuel Marc Gerecht', 'Berna...","['Brett McGurk', 'Barbara Slavin']",NaN,NaN


In [14]:
def get_main_points(fid, main_points):
    """
    Returns main points for both sides given fid and main_points table
    """
    for_points = main_points.loc[main_points['id'] == fid]['For_Main_Points'].item()
    for_points = ast.literal_eval(for_points)
    against_points = main_points.loc[main_points['id'] == fid]['against_Main_Points'].item()
    against_points = ast.literal_eval(against_points)
    return for_points, against_points

In [21]:
punctuations = string.punctuation
def tokenize_point(point):
    """
    Returns a list of lowercased tokens, without punctuations
    """
    tokens = [t for t in word_tokenize(point.lower()) if t not in punctuations]
    return tokens

In [112]:
def word2glove(word, embedding_index):
    """
    Returns GloVe embedding given a word, and zeros if word does not exist in the dictionary
    """
    word = word.lower()
    try:
        word_vec = embedding_index[word]
    except:
        word_vec = np.zeros(embedding_index['the'].shape)
    return word_vec

def point2gloves(point, embedding_index):
    """
    Returns a list of word embeddings given a sentence
    """
    word_vecs = np.array([word2glove(word, embedding_index) for word in tokenize_point(point)])
    return word_vecs
        

In [113]:
def attention_vector(word, embedding_index, title_vecs):
    """
    Returns the attention vector of a word w.r.t. the title (a_k in paper)
    """
    word_vec = word2glove(word, embedding_index)
    weights = cosine_similarity([word_vec], title_vecs)
    attention_vec = np.matmul(title_vecs.T, weights.reshape((-1,1))).T[0]
    return attention_vec

In [104]:
# np.concatenate((attention_vector('king', embedding_index, title_vecs), \
#                word2glove('the', embedding_index)), axis = None)

In [103]:
def matching_vector(attention_vec, word, embedding_index):
    """
    Returns the matching vector of a word w.r.t. the title by concatenating the 
    resulting attention vector and word embedding(m_k in paper)
    """
    matching_vec = np.concatenate((attention_vec, word2glove(word, embedding_index)), axis = None)
    return matching_vec

In [111]:
fid = 'd20191112'
title = main_points.loc[main_points['id'] == fid]['title'].iloc[0]
for_points, against_points = get_main_points(fid, main_points)
print('Debate Title:', title)

Debate Title: Capitalism Is a Blessing


In [117]:
word = 'socialism'
title_vecs = point2gloves(title, embedding_index)
attention_vec = attention_vector(word, embedding_index, title_vecs)
matching_vec = matching_vector(attention_vec, word, embedding_index)

In [130]:
print('word:', word)
print('Similarities between attention vector and words in title:', cosine_similarity([attention_vec], title_vecs))

word: socialism
Similarities between attention vector and words in title: [[0.89212716 0.623381   0.53955615 0.40743887]]


In [118]:
matching_vec.shape

(200,)

In [119]:
matching_vec

array([ 0.3015426 ,  0.9216185 ,  0.9031023 , -0.07591914, -0.00535155,
        0.29403263, -0.47589275, -0.699059  , -0.01627889, -0.251849  ,
       -0.57344663,  0.6658845 ,  0.09023698,  0.04401395,  0.37690887,
       -0.0772756 ,  0.4640494 ,  0.4316342 , -0.51178044, -0.19788025,
        0.33429575, -0.10098885, -0.35101748, -0.24833116,  0.17554505,
        0.41560915, -0.49203956, -0.19756055, -0.11192715, -0.00694678,
       -0.24993534,  1.2522714 , -0.39274085, -0.31340644,  0.26661408,
        0.18764499,  0.09789918, -0.2171377 , -0.40555012, -0.42812616,
       -0.58505064, -0.20553707, -0.2493966 ,  0.50509006, -0.18823734,
       -0.69646555,  0.8033727 ,  0.27810642, -0.3216776 , -0.19422434,
       -0.5528731 ,  0.3381607 , -0.25321126,  1.1384412 , -0.3097309 ,
       -1.947628  ,  0.04279271,  0.34267896,  1.11995   ,  0.01181408,
       -0.04970998, -0.30868697, -0.49640337, -0.45260602,  0.7727301 ,
       -0.0720768 ,  0.6050939 , -0.694137  ,  0.1374662 , -0.68